# **The Impact and Effectivity of Improving Operating Room’s Schedules Using Simulation**

---



Members:
- Alcantara, Alyssa Mae
- Ledesma, Jhon Carlo
- Orellano, Kyla

**Model Development and Simulation**

- This event follows a Discrete-Event Simulation beacuse in this model, some changes might occur on distinct point of time.

Stochastic
- In an operating room, there is a randomness in the arrival and durations of surgery. You also do not know if there is a priority or not in terms of emergency.

Dynamic
- This does not remains constant over time, it evolves. The duration of surgery improves depending on the number of doctors and equipments available in the hospitals.

# **CODE**

---



In [ ]:
!pip install simpy

In [ ]:
import simpy
import random

**parameters**
- *NUM_OR* - number of operating rooms
- *SIM_TIME* - simulation time in minutes (8 hours)
- *TURNOVER_TIME* - cleaning between surgeries
- *EMERGENCY_PROB* - probability of having an emergency surgery

In [ ]:
NUM_OR = 3
SIM_TIME = 480
TURNOVER_TIME = 15
EMERGENCY_PROB = 0.4
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

**surgery duration** (minutes)

In [ ]:
def get_surgery_duration(emergency=False):
    if emergency:
        return random.randint(60, 120)
    else:
        return random.randint(30, 90)

**generation of surgeries** - random time on 15 to 45 mins

In [ ]:
def generate_surgeries(env, or_rooms):
    surgery_id = 0
    while True:
        yield env.timeout(random.randint(15, 45))

        emergency = random.random() < EMERGENCY_PROB
        duration = get_surgery_duration(emergency)
        env.process(perform_surgery(env, surgery_id, or_rooms, duration, emergency))
        surgery_id += 1

**process of surgery and cleaning**

In [ ]:
def perform_surgery(env, surgery_id, or_rooms, duration, emergency):
    with or_rooms.request(priority=0 if emergency else 1) as req:
        arrival_time = env.now
        yield req
        wait_time = env.now - arrival_time

        print(f"{env.now:.1f} min - Surgery {surgery_id} ({'EMERGENCY' if emergency else 'Elective'}) started. Waited {wait_time:.1f} mins.")
        yield env.timeout(duration)
        print(f"{env.now:.1f} min - Surgery {surgery_id} finished. Cleaning OR...")

        yield env.timeout(TURNOVER_TIME)

**run** - there is a priority in emergencies. low no = high prio




In [ ]:
def run_simulation():
    env = simpy.Environment()

    or_rooms = simpy.PriorityResource(env, capacity=NUM_OR)
    env.process(generate_surgeries(env, or_rooms))
    env.run(until=SIM_TIME)

if __name__ == "__main__":
    run_simulation()

35.0 min - Surgery 0 (EMERGENCY) started. Waited 0.0 mins.
58.0 min - Surgery 1 (EMERGENCY) started. Waited 0.0 mins.
96.0 min - Surgery 2 (EMERGENCY) started. Waited 0.0 mins.
126.0 min - Surgery 1 finished. Cleaning OR...
141.0 min - Surgery 3 (Elective) started. Waited 2.0 mins.
142.0 min - Surgery 0 finished. Cleaning OR...
167.0 min - Surgery 4 (EMERGENCY) started. Waited 0.0 mins.
203.0 min - Surgery 2 finished. Cleaning OR...
208.0 min - Surgery 3 finished. Cleaning OR...
218.0 min - Surgery 5 (EMERGENCY) started. Waited 30.0 mins.
223.0 min - Surgery 6 (Elective) started. Waited 20.0 mins.
232.0 min - Surgery 4 finished. Cleaning OR...
247.0 min - Surgery 7 (Elective) started. Waited 9.0 mins.
298.0 min - Surgery 6 finished. Cleaning OR...
303.0 min - Surgery 7 finished. Cleaning OR...
313.0 min - Surgery 8 (Elective) started. Waited 53.0 mins.
316.0 min - Surgery 5 finished. Cleaning OR...
318.0 min - Surgery 9 (Elective) started. Waited 18.0 mins.
340.0 min - Surgery 10 (EMER